In [8]:
#импортируем библиотеку pandas для работы с большими данными
import pandas as pd

In [9]:
#настраиваем отображение датафрейма, убирая ограничения  на ширину и длину столбца
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)

In [131]:
#загружаем наш датасет
df = pd.read_excel('data.xlsx')

In [132]:
# делим каждый текст на токены
df['tokens'] = df['message'].apply(lambda x: x.split())
df['len'] = df['tokens'].apply(lambda x: len(x))
#убираем текста, длиа которых больше 510 токенов
df = df[df['len'] < 510]

In [ ]:
df.drop(columns = ['date_send','sender_role','age','id_room','tokens','len'])

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# загрузка модели трансформера происходит следующим образом

In [2]:
tokenizer1 = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment-rurewiews")
model1 = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment-rurewiews")

In [5]:
tokenizer2 = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment")
model2 = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment")

In [6]:
tokenizer3 = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced")
model3 = AutoModelForSequenceClassification.from_pretrained("cointegrated/rubert-tiny-sentiment-balanced")

In [109]:
def classification(document):
    #в пайплайне указывается задача, название модели и токенизатора, а top_k = 1 говорит о том, что нужно вывести только 1 наиболее подходящий класс
    classifier = pipeline('text-classification', model = model1, tokenizer=tokenizer1, top_k = 1)
    return classifier(document)

In [110]:
df['pred'] = df['message'].apply(lambda x: classification(x))

In [114]:
# проходи по всем значениям столбца "pred" и вытаскиваем всё, что расположено между "label':, '" и "', 'score'"
df['pred'] = df['pred'].apply(lambda x: re.findall(r"label': '([\s\S]+?)', 'score", str(x), flags = re.I))
# поскольку все найденные значения пишутся в квадратных скобках их в итоге тоже необходимо убрать
df['pred'] = df['pred'].apply(lambda x: re.sub("[^A-Za-z0-2_]", "", str(x)))

In [ ]:
df['pred'] = df.pred.replace('neutral', 0).replace('positive', 1).replace('negative', 2)
df['Target'] = df.Target.replace('Neutral', 0).replace('Good', 1).replace('Bad', 2)

In [36]:
from sklearn.metrics import classification_report

In [126]:
#необходимо привести сравниваемые столбцы к числовому формату и задать имена классов
print(classification_report(pd.to_numeric(df['Target']),pd.to_numeric(df['pred']),target_names = ['Neutral','Positive','Negative']))

              precision    recall  f1-score   support

     Neutral       0.32      0.42      0.37       180
    Positive       0.54      0.58      0.56       180
    Negative       0.70      0.44      0.54       180

    accuracy                           0.48       540
   macro avg       0.52      0.48      0.49       540
weighted avg       0.52      0.48      0.49       540



In [ ]:
df1 = df[df_new['Target'] == 0]
df1 = df1[:180]

In [ ]:
df2 = df[df_new['Target'] == 1]
df2 = df2[:180]

In [22]:
df3 = df[df_new['Target'] == 2]
df3 = df3[:180]

In [107]:
new_df = pd.concat([df1, df2, df3], ignore_index=True)

In [ ]:
new_df['pred'] = new_df['message'].apply(lambda x: classification(x))

In [ ]:
# проходи по всем значениям столбца "pred" и вытаскиваем всё, что расположено между "label':, '" и "', 'score'"
new_df['pred'] = new_df['pred'].apply(lambda x: re.findall(r"label': '([\s\S]+?)', 'score", str(x), flags = re.I))
# поскольку все найденные значения пишутся в квадратных скобках их в итоге тоже необходимо убрать
new_df['pred'] = new_df['pred'].apply(lambda x: re.sub("[^A-Za-z0-2_]", "", str(x)))

In [ ]:
new_df['pred'] = new_df.pred.replace('neutral', 0).replace('positive', 1).replace('negative', 2)
new_df['Target'] = new_df.Target.replace('Neutral', 0).replace('Good', 1).replace('Bad', 2)

In [ ]:
print(classification_report(pd.to_numeric(new_df['Target']),pd.to_numeric(new_df['pred']),target_names = ['Neutral','Positive','Negative']))